The suscpetible-infected-recovered (SIR) model is a system of nonlinear ODEs given by
\begin{align}
    \frac{dS}{dt} &= \mu N - \mu S - \eta k I S \\
    \frac{dI}{dt} &= \eta k I S - (\gamma+\mu)I \\
    \frac{dR}{dt} &= \gamma I - \mu R.
    \end{align}

where $\gamma$ is the infection rate, $k$ is the contact rate, $r$ is the recovery rate, and $mu$ represents the collective birth and death rate. Each parameter is assumed to take values in the interval [0,1].


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.stats import uniform, beta, gaussian_kde


In [2]:
# SIR model function
def SIR_model(y, t, params, N):
    # Unpack parameters
    gamma, k, r, mu = params
    # Redefine state variables
    S, I, R = y

    # RHS equations
    dSdt = mu*N - mu*S - gamma*k*I*S
    dIdt = gamma*k*I*S - (r + mu)*I
    dRdt = r*I - mu*R
    return [dSdt, dIdt, dRdt]


In [3]:
# Initial conditions for the SIR model
S0, I0, R0 = 900, 100, 0
X0 = [S0, I0, R0]
N = np.sum(X0)

# Time variables
t_final = 50
dt = 0.05
t_data = np.arange(0, t_final + dt, dt)

ode_options = {'rtol': 1e-6}

# Quantities of interest: integral of the infected state and steady-state equilibrium value of the infected state
QoI_time = len(t_data)
Y0 = np.array([S0, I0, R0])

# Identify upper and lower bounds for [gamma, k, r, mu]
parameter_names = ['gamma', 'k', 'r', 'mu']
UB = np.array([0.3, 1.0, 1.0, 0.5])
LB = np.array([0.1, 0.1, 0.1, 0.1])

par_all = 0.5 * (UB + LB)

# Parameters
par_nom = par_all
pars = par_all
num_par = len(UB)

# Number of samples, parameters, levels, and step size
R = 100
p = num_par
l = 60
delta = l / (2 * (l - 1))
upper = UB
lower = LB
d = np.zeros((R, p))

# Randomization algorithm
A = np.zeros((p + 1, p))
for i in range(p):
    A[i + 1:p + 1, i] = np.ones(p - i)
    
X = np.zeros(((p + 1) * R, p * R))
F_storage = [[None for _ in range(R)] for _ in range(p + 1)]
qstar = np.random.uniform(0, 1, (R, p))
Jp = np.ones((p + 1, p))
J1 = np.ones((p + 1, 1))
P = np.eye(p)
UL_MAT = np.eye(p) * (upper - lower)



In [ ]:
## Now, calculate the Morris' indices
func_evals = 1
for i in range(R):
    qcurr = qstar[i, :]
    pm1 = np.random.rand(p)
    Dstar = np.eye(p) * (pm1 > 0.5) - np.eye(p) * (pm1 <= 0.5)
    where = np.argsort(np.random.rand(p))
    Pstar = P[where, :]
    Astar = np.multiply(J1, qcurr) + (delta / 2) * (((2 * A - Jp) @ Dstar + Jp) @ Pstar)
    C = J1 * lower + Astar @ UL_MAT
    fpast = odeint(SIR_model, Y0, t_data, args=(C[0], N), **ode_options)
    F_storage[0][i] = fpast
    for j in range(p):
        fstep = odeint(SIR_model, Y0, t_data, args=(C[j + 1], N), **ode_options)
        par_sign = np.sign(C[j + 1, where[j]] - C[j, where[j]])
        d[i, where[j]] = par_sign * (fstep[-1, 2] - fpast[-1, 2]) / delta
        fpast = fstep
        F_storage[where[j] + 1][i] = fpast

mu = np.mean(d, axis=0)
mu_star = np.mean(np.abs(d), axis=0)
sigma = np.sqrt(np.sum((d - mu_star)**2, axis=0) / (R - 1))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 1)

In [ ]:
plt.figure()
plt.hold(True)
for i in range(num_par):
    plt.plot(mu_star[i], sigma[i], 'k.', markersize=20)
    plt.text(mu_star[i] * 1.01, sigma[i], parameter_names[i], fontsize=16)
plt.hold(False)
plt.show()

rank = np.sqrt(mu_star**2 + sigma**2)
plt.figure()
plt.bar(range(len(rank)), rank)
plt.xticks(range(len(rank)), parameter_names)
plt.show()